In [1]:
import os
import gc
import torch
os.chdir("/work/pi_jensen_umass_edu/jnainani_umass_edu/CircuitAnalysisSAEs")
import json
from sae_lens import SAE, HookedSAETransformer
from circ4latents import data_gen
from functools import partial
import einops

# Function to manage CUDA memory and clean up
def cleanup_cuda():
   torch.cuda.empty_cache()
   gc.collect()
# cleanup_cuda()
# Load the config
with open("config.json", 'r') as file:
   config = json.load(file)
token = config.get('huggingface_token', None)
os.environ["HF_TOKEN"] = token

# Define device
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Device: {device}")

hf_cache = "/work/pi_jensen_umass_edu/jnainani_umass_edu/mechinterp/huggingface_cache/hub"
os.environ["HF_HOME"] = hf_cache

# Load the model
model = HookedSAETransformer.from_pretrained("google/gemma-2-9b", device=device, cache_dir=hf_cache)



Device: cuda


Loading checkpoint shards:   0%|          | 0/8 [00:00<?, ?it/s]

Loaded pretrained model google/gemma-2-9b into HookedTransformer


In [2]:
sae21 = SAE.from_pretrained(release="gemma-scope-9b-pt-res", sae_id=f"layer_21/width_16k/average_l0_36", device=device)[0]

In [3]:
from transformer_lens.utils import test_prompt
prompt = """>>> my_dict = {"a": 1, "b": 2}
>>> var = my_dict["a"]
"""
test_prompt(prompt, "1", model)

Tokenized prompt: ['<bos>', '>>>', ' my', '_', 'dict', ' =', ' {"', 'a', '":', ' ', '1', ',', ' "', 'b', '":', ' ', '2', '}', '\n', '>>>', ' var', ' =', ' my', '_', 'dict', '["', 'a', '"]', '\n']
Tokenized answer: [' ', '1']


Performance on answer token:
Rank: 16       Logit: -1.16 Prob:  0.02% Token: | |

Top 0th token. Logit:  7.59 Prob: 98.11% Token: |>>>|
Top 1th token. Logit:  2.14 Prob:  0.42% Token: |Traceback|
Top 2th token. Logit:  1.77 Prob:  0.29% Token: |1|
Top 3th token. Logit:  1.00 Prob:  0.13% Token: |#|
Top 4th token. Logit:  0.75 Prob:  0.10% Token: |...|
Top 5th token. Logit:  0.55 Prob:  0.09% Token: |2|
Top 6th token. Logit:  0.48 Prob:  0.08% Token: |var|
Top 7th token. Logit:  0.43 Prob:  0.08% Token: |<eos>|
Top 8th token. Logit: -0.07 Prob:  0.05% Token: |>>|
Top 9th token. Logit: -0.22 Prob:  0.04% Token: | >>>|


Performance on answer token:
Rank: 1        Logit: 24.75 Prob: 23.78% Token: |1|

Top 0th token. Logit: 25.75 Prob: 64.97% Token: |
|
Top 1th token. Logit: 24.75 Prob: 23.78% Token: |1|
Top 2th token. Logit: 23.44 Prob:  6.42% Token: |

|
Top 3th token. Logit: 22.35 Prob:  2.16% Token: |2|
Top 4th token. Logit: 20.85 Prob:  0.48% Token: |


|
Top 5th token. Logit: 20.67 Prob:  0.40% Token: |3|
Top 6th token. Logit: 20.16 Prob:  0.24% Token: |	|
Top 7th token. Logit: 20.13 Prob:  0.24% Token: |5|
Top 8th token. Logit: 19.71 Prob:  0.15% Token: |4|
Top 9th token. Logit: 19.69 Prob:  0.15% Token: |0|


Ranks of the answer tokens: [(' ', 16), ('1', 1)]

# Simple Key Error Detection

# Key Error Detection with Duplicate

In [8]:

import json
import random

# Expanding the name pool with a larger set of names
extended_name_pool = [
    "Bob", "Sam", "Lilly", "Rob", "Alice", "Charlie", "Sally", "Tom", "Jake", "Emily", 
    "Megan", "Chris", "Sophia", "James", "Oliver", "Isabella", "Mia", "Jackson", 
    "Emma", "Ava", "Lucas", "Benjamin", "Ethan", "Grace", "Olivia", "Liam", "Noah"
]

for name in extended_name_pool:
    assert len(model.tokenizer.encode(name)) == 2, f"Name {name} has more than 1 token"

# Function to generate the dataset with correct and incorrect keying into dictionaries
def generate_extended_dataset(name_pool, num_samples=5):
    dataset = []
    
    for _ in range(num_samples):
        # Randomly select 5 names from the pool
        selected_names = random.sample(name_pool, 5)
        # Assign random ages to the selected names
        age_dict = {name: random.randint(10, 19) for name in selected_names}
        
        # Create a correct example
        correct_name = random.choice(list(age_dict.keys()))
        correct_prompt = f'Type "help", "copyright", "credits" or "license" for more information.\n>>> age = {age_dict}\n>>> var = age["{correct_name}"]\n'
        correct_response = age_dict[correct_name]
        correct_token = str(correct_response)[0]
        
        # Create an incorrect example with a name not in the dictionary
        incorrect_name = random.choice([name for name in name_pool if name not in age_dict])
        incorrect_prompt = f'Type "help", "copyright", "credits" or "license" for more information.\n>>> age = {age_dict}\n>>> var = age["{incorrect_name}"]\n'
        incorrect_response = "Traceback"
        incorrect_token = "Traceback"
        
        # Append the pair of correct and incorrect examples
        dataset.append({
            "correct": {
                "prompt": correct_prompt,
                "response": correct_response,
                "token": correct_token
            },
            "error": {
                "prompt": incorrect_prompt,
                "response": incorrect_response,
                "token": incorrect_token
            }
        })
        
    return dataset

# Generate the extended dataset
json_dataset = generate_extended_dataset(extended_name_pool, num_samples=100)

# Output the JSON structure

# %%
clean_prompts = []
corr_prompts = []

answer_token = model.to_single_token(">>>")
traceback_token = model.to_single_token("Traceback")

for item in json_dataset[:50]:
    corr_prompts.append(item["correct"]["prompt"])
    clean_prompts.append(item["error"]["prompt"])

clean_tokens = model.to_tokens(clean_prompts)
corr_tokens = model.to_tokens(corr_prompts)

# %%
def logit_diff_fn(logits):
    err = logits[:, -1, traceback_token]
    no_err = logits[:, -1, answer_token]
    return (err - no_err).mean()

# Disable gradients for all parameters
for param in model.parameters():
   param.requires_grad_(False)

# # Compute logits for clean and corrupted samples
logits = model(clean_tokens)
clean_diff = logit_diff_fn(logits)

logits = model(corr_tokens)
corr_diff = logit_diff_fn(logits)

print(f"clean_diff: {clean_diff}")
print(f"corr_diff: {corr_diff}")

# # Cleanup
del logits
cleanup_cuda()

# # Define error type metric
def _err_type_metric(logits, clean_logit_diff, corr_logit_diff):
    patched_logit_diff = logit_diff_fn(logits)
    return (patched_logit_diff - corr_logit_diff) / (clean_logit_diff - corr_logit_diff)

err_metric_denoising = partial(_err_type_metric, clean_logit_diff=clean_diff, corr_logit_diff=corr_diff)

clean_diff: 3.590031623840332
corr_diff: -4.768364429473877


In [11]:
import json
import random

# Expanding the name pool with a larger set of names
extended_name_pool = [
    "Bob", "Sam", "Lilly", "Rob", "Alice", "Charlie", "Sally", "Tom", "Jake", "Emily", 
    "Megan", "Chris", "Sophia", "James", "Oliver", "Isabella", "Mia", "Jackson", 
    "Emma", "Ava", "Lucas", "Benjamin", "Ethan", "Grace", "Olivia", "Liam", "Noah"
]

for name in extended_name_pool:
    assert len(model.tokenizer.encode(name)) == 2, f"Name {name} has more than 1 token"

# Function to generate the dataset for Level 3 - dicts with duplicate
def generate_duplicate_key_dataset(name_pool, num_samples=5):
    dataset = []
    
    for _ in range(num_samples):
        # Randomly select 5 names from the pool
        selected_names = random.sample(name_pool, 5)
        # Create a dictionary where some values are also keys
        value_keys = random.sample(selected_names, 2)  # Choose 2 names to act as both keys and values
        random_values = random.choices(name_pool, k=5)
        age_dict = {key: value_keys[i % len(value_keys)] if i < len(value_keys) else random_values[i]
                    for i, key in enumerate(selected_names)}

        # Create a correct example
        correct_key = random.choice(list(age_dict.keys()))
        correct_prompt = f'Type "help", "copyright", "credits" or "license" for more information.\n>>> mapping = {age_dict}\n>>> var = mapping["{correct_key}"]\n'
        correct_response = age_dict[correct_key]
        correct_token = model.to_single_token(str(correct_response))

        # Create an incorrect example by attempting to access a value as if it's a key
        incorrect_value = random.choice([value for value in age_dict.values() if value not in age_dict])
        incorrect_prompt = f'Type "help", "copyright", "credits" or "license" for more information.\n>>> mapping = {age_dict}\n>>> var = mapping["{incorrect_value}"]\n'
        incorrect_response = "Traceback"
        incorrect_token = "Traceback"
        
        # Append the pair of correct and incorrect examples
        dataset.append({
            "correct": {
                "prompt": correct_prompt,
                "response": correct_response,
                "token": correct_token
            },
            "error": {
                "prompt": incorrect_prompt,
                "response": incorrect_response,
                "token": incorrect_token
            }
        })
        
    return dataset

# Generate the duplicate key dataset
json_dataset = generate_duplicate_key_dataset(extended_name_pool, num_samples=10)
print(json_dataset[0]['correct']['prompt'])


Type "help", "copyright", "credits" or "license" for more information.
>>> mapping = {'Charlie': 'Grace', 'Ava': 'Chris', 'Sally': 'Isabella', 'Chris': 'Chris', 'Grace': 'Noah'}
>>> var = mapping["Sally"]



In [13]:
print(json_dataset[0]['error']['prompt'])

Type "help", "copyright", "credits" or "license" for more information.
>>> mapping = {'Charlie': 'Grace', 'Ava': 'Chris', 'Sally': 'Isabella', 'Chris': 'Chris', 'Grace': 'Noah'}
>>> var = mapping["Isabella"]



In [13]:
from transformer_lens.utils import test_prompt
prompt = """Type "help", "copyright", "credits" or "license" for more information.
>>> mapping = {'Charlie': 'Grace', 'Ava': 'Chris', 'Sally': 'Isabella', 'Grace': 'Noah'}
>>> print(mapping["Isabella"])
"""
test_prompt(prompt, "Traceback", model, prepend_space_to_answer=False)

Tokenized prompt: ['<bos>', 'Type', ' "', 'help', '",', ' "', 'copyright', '",', ' "', 'credits', '"', ' or', ' "', 'license', '"', ' for', ' more', ' information', '.', '\n', '>>>', ' mapping', ' =', " {'", 'Charlie', "':", " '", 'Grace', "',", " '", 'Ava', "':", " '", 'Chris', "',", " '", 'Sally', "':", " '", 'Isabella', "',", " '", 'Grace', "':", " '", 'Noah', "'}", '\n', '>>>', ' print', '(', 'mapping', '["', 'Isabella', '"])', '\n']
Tokenized answer: ['Traceback']


Performance on answer token:
Rank: 7        Logit: 25.29 Prob:  2.61% Token: |Traceback|

Top 0th token. Logit: 27.90 Prob: 35.36% Token: |Sally|
Top 1th token. Logit: 27.34 Prob: 20.17% Token: |Grace|
Top 2th token. Logit: 26.75 Prob: 11.27% Token: |Noah|
Top 3th token. Logit: 26.36 Prob:  7.60% Token: |>>>|
Top 4th token. Logit: 25.69 Prob:  3.90% Token: |Ava|
Top 5th token. Logit: 25.63 Prob:  3.64% Token: |Isabella|
Top 6th token. Logit: 25.29 Prob:  2.61% Token: |Chris|
Top 7th token. Logit: 25.29 Prob:  2.61% Token: |Traceback|
Top 8th token. Logit: 25.11 Prob:  2.18% Token: |Charlie|
Top 9th token. Logit: 24.68 Prob:  1.42% Token: |'|


Ranks of the answer tokens: [('Traceback', 7)]

In [21]:
from transformer_lens.utils import test_prompt
prompt = """Type "help", "copyright", "credits" or "license" for more information.
>>> mapping = {'a': 'b', 'c': 'd', 'e': 'f'}
>>> print(mapping["f"])
"""
test_prompt(prompt, "Traceback", model, prepend_space_to_answer=False)

Tokenized prompt: ['<bos>', 'Type', ' "', 'help', '",', ' "', 'copyright', '",', ' "', 'credits', '"', ' or', ' "', 'license', '"', ' for', ' more', ' information', '.', '\n', '>>>', ' mapping', ' =', " {'", 'a', "':", " '", 'b', "',", " '", 'c', "':", " '", 'd', "',", " '", 'e', "':", " '", 'f', "'}", '\n', '>>>', ' print', '(', 'mapping', '["', 'f', '"])', '\n']
Tokenized answer: ['Traceback']


Performance on answer token:
Rank: 0        Logit: 27.02 Prob: 50.37% Token: |Traceback|

Top 0th token. Logit: 27.02 Prob: 50.37% Token: |Traceback|
Top 1th token. Logit: 25.30 Prob:  9.04% Token: |e|
Top 2th token. Logit: 25.22 Prob:  8.39% Token: |>>>|
Top 3th token. Logit: 24.76 Prob:  5.24% Token: |Syntax|
Top 4th token. Logit: 24.47 Prob:  3.94% Token: |f|
Top 5th token. Logit: 24.17 Prob:  2.92% Token: |None|
Top 6th token. Logit: 23.95 Prob:  2.35% Token: |b|
Top 7th token. Logit: 23.88 Prob:  2.19% Token: |d|
Top 8th token. Logit: 23.47 Prob:  1.46% Token: |a|
Top 9th token. Logit: 23.28 Prob:  1.20% Token: |'|


Ranks of the answer tokens: [('Traceback', 0)]

In [14]:
from transformer_lens.utils import test_prompt
prompt = """Type "help", "copyright", "credits" or "license" for more information.
>>> my_dict = {'Charlie': 'Grace', 'Ava': 'Chris', 'Sally': 'Isabella', 'Grace': 'Noah'}
>>> for key in ["Charlie", "Ava", "Isabella"]:
...     print(my_dict[key])
... 
Grace
Chris
"""
test_prompt(prompt, "Traceback", model, prepend_space_to_answer=False)

Tokenized prompt: ['<bos>', 'Type', ' "', 'help', '",', ' "', 'copyright', '",', ' "', 'credits', '"', ' or', ' "', 'license', '"', ' for', ' more', ' information', '.', '\n', '>>>', ' my', '_', 'dict', ' =', " {'", 'Charlie', "':", " '", 'Grace', "',", " '", 'Ava', "':", " '", 'Chris', "',", " '", 'Sally', "':", " '", 'Isabella', "',", " '", 'Grace', "':", " '", 'Noah', "'}", '\n', '>>>', ' for', ' key', ' in', ' ["', 'Charlie', '",', ' "', 'Ava', '",', ' "', 'Isabella', '"]:', '\n', '...', '     ', 'print', '(', 'my', '_', 'dict', '[', 'key', '])', '\n', '...', ' ', '\n', 'Grace', '\n', 'Chris', '\n']
Tokenized answer: ['Traceback']


Performance on answer token:
Rank: 15       Logit: 22.56 Prob:  0.15% Token: |Traceback|

Top 0th token. Logit: 28.63 Prob: 64.16% Token: |Noah|
Top 1th token. Logit: 26.97 Prob: 12.28% Token: |Isabella|
Top 2th token. Logit: 26.55 Prob:  8.05% Token: |Grace|
Top 3th token. Logit: 26.08 Prob:  5.01% Token: |Sally|
Top 4th token. Logit: 25.12 Prob:  1.93% Token: |>>>|
Top 5th token. Logit: 25.01 Prob:  1.72% Token: |Chris|
Top 6th token. Logit: 24.82 Prob:  1.43% Token: |None|
Top 7th token. Logit: 23.97 Prob:  0.61% Token: |'|
Top 8th token. Logit: 23.85 Prob:  0.54% Token: |No|
Top 9th token. Logit: 23.69 Prob:  0.46% Token: |...|


Ranks of the answer tokens: [('Traceback', 15)]

In [6]:
mapping = {'Charlie': 'Grace', 'Ava': 'Chris', 'Sally': 'Isabella', 'Grace': 'Noah'}
var = mapping["Isabella"]

KeyError: 'Isabella'

In [27]:
import json
import random

# Expanding the name pool with a larger set of names
extended_name_pool = [
    "Bob", "Sam", "Lilly", "Rob", "Alice", "Charlie", "Sally", "Tom", "Jake", "Emily", 
    "Megan", "Chris", "Sophia", "James", "Oliver", "Isabella", "Mia", "Jackson", 
    "Emma", "Ava", "Lucas", "Benjamin", "Ethan", "Grace", "Olivia", "Liam", "Noah"
]

for name in extended_name_pool:
    assert len(model.tokenizer.encode(name)) == 2, f"Name {name} has more than 1 token"
# Function to generate the dataset for Level 2 - nested dictionaries
# Function to generate the dataset for Level 2 - nested dictionaries with unique inner keys per outer key
def generate_nested_dict_dataset_unique_inner_keys(name_pool, num_samples=5):
    dataset = []
    
    for _ in range(num_samples):
        # Randomly select 5 outer keys
        outer_keys = random.sample(name_pool, 3)
        
        # Create a nested dictionary where each outer key has a unique set of inner keys
        nested_dict = {
            outer_key: {inner_key: random.randint(10, 19) for inner_key in random.sample(name_pool, 2)}
            for outer_key in outer_keys
        }
        
        # Create a correct example
        correct_outer_key = random.choice(list(nested_dict.keys()))
        correct_inner_key = random.choice(list(nested_dict[correct_outer_key].keys()))
        correct_prompt = (
            f'Type "help", "copyright", "credits" or "license" for more information.\n'
            f'>>> data = {nested_dict}\n>>> var = data["{correct_outer_key}"]["{correct_inner_key}"]\n'
        )
        correct_response = nested_dict[correct_outer_key][correct_inner_key]
        correct_token = str(correct_response)[0]
        
        # Create an incorrect example with a non-existent inner key for an existing outer key
        incorrect_outer_key = correct_outer_key
        incorrect_inner_key = random.choice(
            [key for key in name_pool if key not in nested_dict[incorrect_outer_key]]
        )
        incorrect_prompt = (
            f'Type "help", "copyright", "credits" or "license" for more information.\n'
            f'>>> data = {nested_dict}\n>>> var = data["{incorrect_outer_key}"]["{incorrect_inner_key}"]\n'
        )
        incorrect_response = "Traceback"
        incorrect_token = "Traceback"
        
        # Append the pair of correct and incorrect examples
        dataset.append({
            "correct": {
                "prompt": correct_prompt,
                "response": correct_response,
                "token": correct_token
            },
            "error": {
                "prompt": incorrect_prompt,
                "response": incorrect_response,
                "token": incorrect_token
            }
        })
        
    return dataset

# Generate the nested dictionary dataset
json_dataset = generate_nested_dict_dataset_unique_inner_keys(extended_name_pool, num_samples=100)

print(json_dataset[0]['correct']['prompt'])


Type "help", "copyright", "credits" or "license" for more information.
>>> data = {'Grace': {'Emma': 14, 'Megan': 11}, 'Bob': {'Lucas': 19, 'Emma': 19}, 'Olivia': {'Isabella': 13, 'Sally': 14}}
>>> var = data["Grace"]["Emma"]

